<a href="https://colab.research.google.com/github/Maha9-lakshmi/skinlesion_telebot/blob/main/Skin_Lesion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Creating Token from kaggle and Upload in Colab

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp /content/kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

#Downloading the dataset from kaggle

In [ ]:
!kaggle datasets download -d dipuiucse/monkeypoxskinimagedataset

100% 54.0M/54.1M [00:01<00:00, 65.7MB/s]
100% 54.1M/54.1M [00:01<00:00, 53.6MB/s]


In [ ]:
!unzip -q monkeypoxskinimagedataset.zip -d monkeypox

In [ ]:
!ls monkeypox

'Monkeypox Skin Image Dataset'


#Model building with MobileNetV2 and Preprocessing

In [ ]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.applications import MobileNetV2
from keras.preprocessing.image import ImageDataGenerator

# Path to the dataset directory (each class should have its own subdirectory)
dataset_dir = '/content/monkeypox/Monkeypox Skin Image Dataset'

# Define the classes (folder names in the dataset directory)
classes = ['Chickenpox', 'Measles', 'Monkeypox', 'Normal']

# Function to load and preprocess the data
def load_data():
    data = []
    labels = []
    for class_id, class_name in enumerate(classes):
        class_dir = os.path.join(dataset_dir, class_name)
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))  # Resize images to a fixed size
            data.append(img)
            labels.append(class_id)

    data = np.array(data, dtype='float32') / 255.0  # Normalize pixel values to [0, 1]
    labels = np.array(labels)
    return data, labels

# Load the data
X, y = load_data()

# Convert labels to one-hot encoded vectors
y = to_categorical(y, num_classes=len(classes))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X_train)

# Transfer learning with MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom head for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(classes), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers for transfer learning
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with fine-tuning
epochs = 35
steps_per_epoch = len(X_train) // 32
validation_steps = len(X_test) // 32

model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=epochs, steps_per_epoch=steps_per_epoch,
          validation_data=(X_test, y_test), validation_steps=validation_steps)

# Save the trained model
model.save('monkeypox_classification_model.h5')


9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/35
19/19 [==============================] - 19s 456ms/step - loss: 1.0569 - accuracy: 0.6336 - val_loss: 0.5551 - val_accuracy: 0.7922
Epoch 2/35
19/19 [==============================] - 6s 324ms/step - loss: 0.6645 - accuracy: 0.7568 - val_loss: 0.5652 - val_accuracy: 0.7727
Epoch 3/35
19/19 [==============================] - 8s 460ms/step - loss: 0.5331 - accuracy: 0.7928 - val_loss: 0.4421 - val_accuracy: 0.8312
Epoch 4/35
19/19 [==============================] - 7s 357ms/step - loss: 0.4325 - accuracy: 0.8579 - val_loss: 0.4349 - val_accuracy: 0.8442
Epoch 5/35
19/19 [==============================] - 6s 327ms/step - loss: 0.4073 - accuracy: 0.8510 - val_loss: 0.4512 - val_accuracy: 0.8506
Epoch 6/35
19/19 [==============================] - 7s 386ms/step - loss: 0.3581 - accuracy: 0.8699 - val_loss: 0.4295 - val_accuracy: 0.8312
Epoch 7/35
19/19 [==============================] - 7s 392ms/step - loss: 0.3827 - a

# Telebot Integration

In [ ]:
!pip install opencv-python-headless
!pip install keras
!pip install pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.12.0-py3-none-any.whl size=213954 sha256=24ef6ebf2748ed1d3e98cc8e9fa9cbb7af749dba51db4236059cd8f1679532cd
  Stored in directory: /root/.cache/pip/wheels/7e/ba/82/f3ab5bc48525778633bccc741c0424677ed3435736221819f4
Successfully built pyTelegramBotAPI


In [ ]:
import numpy as np
import os
import cv2
import telebot
from keras.models import load_model
from keras.preprocessing import image
from io import BytesIO

# Initialize Telegram Bot
bot_token = "6333386994:AAHRSqoaTQEtjhJZhhWGs7S6WP6i99-zByI"
bot = telebot.TeleBot(bot_token)

# Path to the trained model
model_path = 'monkeypox_classification_model.h5'
model = load_model(model_path)

# Define class labels
classes = ['Chickenpox', 'Measles', 'Monkeypox', 'Normal']

# Function to preprocess the image and make a prediction
def preprocess_image(image_data):
    img = cv2.imdecode(np.frombuffer(image_data, np.uint8), cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img.astype('float32') / 255.0
    return img

def classify_image(image_data):
    img = preprocess_image(image_data)
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    predicted_class_idx = np.argmax(prediction)
    predicted_class = classes[predicted_class_idx]
    confidence = prediction[0][predicted_class_idx] * 100
    return predicted_class, confidence

# Telegram bot message handler for images sent as file attachments
@bot.message_handler(content_types=['photo'])
def handle_image(message):
    try:
        # Get the file ID of the largest photo (last element)
        file_id = message.photo[-1].file_id
        # Download the image
        file_info = bot.get_file(file_id)
        image_data = bot.download_file(file_info.file_path)
        # Process the image and make a prediction
        predicted_class, confidence = classify_image(image_data)
        # Send the prediction back to the user
        response = f"Prediction: {predicted_class}\nConfidence: {confidence:.2f}%"
        bot.reply_to(message, response)
    except Exception as e:
        bot.reply_to(message, f"Error: {str(e)}")

# Telegram bot command handler
@bot.message_handler(commands=['start', 'help'])
def handle_start_help(message):
    bot.reply_to(message, "Send me an image to classify using the /classify command.")

# Start the bot
print("Bot is running...")
bot.polling()

Bot is running...
1/1 [==============================] - 0s 22ms/step
